# Description

The goal of this notebook is very simple: it just lists the content (traits/diseases) that belong to each cluster across all selected "best partitions". Although one would take a look at them here to check whether clusters of traits make sense, that analysis is carried out first by looking at the clustering trees (which are generated later). Then, this notebooks serves as a simple list with the content of the clusters.

# Modules loading

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from IPython.display import display
from pathlib import Path

import pandas as pd

from utils import generate_result_set_name
import conf

# Settings

In [3]:
CONSENSUS_CLUSTERING_DIR = Path(
    conf.RESULTS["CLUSTERING_DIR"], "consensus_clustering"
).resolve()

display(CONSENSUS_CLUSTERING_DIR)

PosixPath('/home/miltondp/projects/labs/greenelab/phenoplier/base3/results/clustering/consensus_clustering')

## Load data

In [4]:
INPUT_SUBSET = "umap"

In [5]:
INPUT_STEM = "z_score_std-projection-smultixcan-efo_partial-mashr-zscores"

In [6]:
DR_OPTIONS = {
    "n_components": 5,
    "metric": "euclidean",
    "n_neighbors": 15,
    "random_state": 0,
}

In [7]:
input_filepath = Path(
    conf.RESULTS["DATA_TRANSFORMATIONS_DIR"],
    INPUT_SUBSET,
    generate_result_set_name(
        DR_OPTIONS, prefix=f"{INPUT_SUBSET}-{INPUT_STEM}-", suffix=".pkl"
    ),
).resolve()
display(input_filepath)

assert input_filepath.exists(), "Input file does not exist"

input_filepath_stem = input_filepath.stem
display(input_filepath_stem)

PosixPath('/home/miltondp/projects/labs/greenelab/phenoplier/base3/results/data_transformations/umap/umap-z_score_std-projection-smultixcan-efo_partial-mashr-zscores-metric_euclidean-n_components_5-n_neighbors_15-random_state_0.pkl')

'umap-z_score_std-projection-smultixcan-efo_partial-mashr-zscores-metric_euclidean-n_components_5-n_neighbors_15-random_state_0'

In [8]:
data_umap = pd.read_pickle(input_filepath)

In [9]:
data_umap.shape

(3749, 5)

In [10]:
data_umap.head()

,UMAP1,UMAP2,UMAP3,UMAP4,UMAP5
100001_raw-Food_weight,-8.437452,1.664409,9.233617,6.944210,16.781176
100002_raw-Energy,-8.388347,1.670845,9.251021,6.975690,16.698511
100003_raw-Protein,-8.424020,1.696144,9.226449,6.970763,16.768959
100004_raw-Fat,-8.393674,1.685311,9.258855,6.992673,16.728815
100005_raw-Carbohydrate,-8.427763,1.685172,9.234971,6.967204,16.751522


# Load best partitions

In [11]:
input_file = Path(CONSENSUS_CLUSTERING_DIR, "best_partitions_by_k.pkl").resolve()
display(input_file)

PosixPath('/home/miltondp/projects/labs/greenelab/phenoplier/base3/results/clustering/consensus_clustering/best_partitions_by_k.pkl')

In [12]:
best_partitions = pd.read_pickle(input_file)

In [13]:
best_partitions.shape

(59, 4)

In [14]:
best_partitions.head()

,method,partition,ami_mean,selected
k,,,,
11,scc_020,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.301839,True
10,scc_020,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.301738,True
9,scc_020,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.301301,True
4,eac_complete_coassoc_matrix,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.300385,True
8,scc_020,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.300171,True


# Analysis of clusterings

In [15]:
from IPython.display import HTML

In [16]:
def show_cluster_stats(clustering_data, selected_partition, selected_cluster):
    traits = [t for t in clustering_data[selected_partition == selected_cluster].index]
    display(len(traits))
    display(traits)

In [17]:
selected_k_values = best_partitions[best_partitions["selected"]].index.tolist()
selected_k_values.sort()
display(selected_k_values)

[4, 5, 7, 8, 9, 10, 11, 14, 22, 24, 26, 28, 38, 41, 45]

In [18]:
for k in selected_k_values:
    display(HTML(f"<h2>Partition with k={k}</h2>"))
    display(best_partitions.loc[k])

    part = best_partitions.loc[k, "partition"]
    display(part.shape)

    part_stats = pd.Series(part).value_counts()
    display(part_stats)

    for cluster_number in part_stats.index.sort_values():
        display(HTML(f"<h3>Cluster {k}.{cluster_number}</h3>"))

        cluster_traits = data_umap[part == cluster_number].index
        display(len(cluster_traits))
        display(cluster_traits)

method                             eac_complete_coassoc_matrix
partition    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
ami_mean                                              0.300385
selected                                                  True
Name: 4, dtype: object

(3749,)

0    3648
2      41
3      40
1      20
dtype: int64

3648

Index(['100001_raw-Food_weight', '100002_raw-Energy', '100003_raw-Protein',
       '100004_raw-Fat', '100005_raw-Carbohydrate', '100006_raw-Saturated_fat',
       '100007_raw-Polyunsaturated_fat', '100008_raw-Total_sugars',
       '100009_raw-Englyst_dietary_fibre', '100010-Portion_size',
       ...
       'visual impairment', 'vitiligo', 'vitreous body disease',
       'vocal cord polyp', 'voice disorders',
       'wellbeing measurement AND family relationship', 'wheezing',
       'whooping cough', 'worry measurement', 'wrist fracture'],
      dtype='object', length=3648)

...

Index(['189_raw-Townsend_deprivation_index_at_recruitment',
       '24003_raw-Nitrogen_dioxide_air_pollution_2010',
       '24004_raw-Nitrogen_oxides_air_pollution_2010',
       '24005_raw-Particulate_matter_air_pollution_pm10_2010',
       '24006_raw-Particulate_matter_air_pollution_pm25_2010',
       '24007_raw-Particulate_matter_air_pollution_pm25_absorbance_2010',
       '24009-Traffic_intensity_on_the_nearest_road',
       '24012_raw-Inverse_distance_to_the_nearest_major_road',
       '24013-Total_traffic_load_on_major_roads', '24014-Close_to_major_road',
       '24015-Sum_of_road_length_of_major_roads_within_100m',
       '24016_raw-Nitrogen_dioxide_air_pollution_2005',
       '24017_raw-Nitrogen_dioxide_air_pollution_2006',
       '24018_raw-Nitrogen_dioxide_air_pollution_2007',
       '24019_raw-Particulate_matter_air_pollution_pm10_2007',
       '24020_raw-Average_daytime_sound_level_of_noise_pollution',
       '24021_raw-Average_evening_sound_level_of_noise_pollution',
      

8

Index(['20127_raw-Neuroticism_score', 'anxiety', 'feeling tense measurement',
       'mood swings', 'nervousness',
       'seeing a general practitioner for nerves, anxiety, tension or depression, self-reported',
       'tiredness measurement', 'worry measurement'],
      dtype='object')

1

Index(['30070_raw-Red_blood_cell_erythrocyte_distribution_width'], dtype='object')